In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('train.csv')
labels = df['avg_rating']
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df['JokeSentences'])
X_tfidf_dense = pd.DataFrame(X_tfidf.todense(), columns=vectorizer.get_feature_names_out())
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf_dense, labels, test_size=0.2, random_state=42
)

knn_regressor = KNeighborsRegressor(n_neighbors =  3 , metric = 'euclidean')
knn_regressor.fit(X_train, y_train)

y_pred_knn = knn_regressor.predict(X_test)
r2_knn = r2_score(y_test, y_pred_knn)
print(f"KNN R-squared: {r2_knn}")

KNN R-squared: 0.8085019173791494


In [3]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11 ],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}


knn_regressor = KNeighborsRegressor()


grid_search = GridSearchCV(knn_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)


grid_search.fit(X_train, y_train)


print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative Mean Squared Error):", grid_search.best_score_)


best_knn_regressor = grid_search.best_estimator_
y_pred = best_knn_regressor.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test Set Mean Squared Error:", test_mse)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best Score (Negative Mean Squared Error): -0.5903242347787284
Test Set Mean Squared Error: 0.35814641678922976


In [9]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)
r2 = r2_score(y_test, y_pred_lr)
print("R² Score for Linear Regression:", r2)

R² Score for Linear Regression: 0.8167202284017401


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
model = SVR(kernel = 'linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R² Score for SVR:", r2)

R² Score for SVR: 0.7169882598292873


In [6]:
with open('knn_regressor.pkl', 'wb') as file:
    pickle.dump(knn_regressor, file)

with open('linear_regression_model.pkl', 'wb') as file:
    pickle.dump(lr_model, file) 

with open('svr_model.pkl', 'wb') as file:
    pickle.dump(model, file)  

In [10]:
mse_knn = mean_squared_error(y_test, y_pred_knn)
print(f"KNN Regressor MSE: {mse_knn}")

mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression MSE: {mse_lr}")

mse_svr = mean_squared_error(y_test, y_pred)
print(f"SVR MSE: {mse_svr}")

KNN Regressor MSE: 0.4096690202659405
Linear Regression MSE: 0.3920877088565129
SVR MSE: 0.6054428364645724
